In [1]:
import glob
import os
import shutil

shutil.rmtree("articles", ignore_errors=True)
os.makedirs("articles", exist_ok=True)

from script.cropPage import *
from script.textMining import *
from script.textSegments import *

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
import ollama

ollama.pull("llama3")
ollama.pull("nomic-embed-text")

{'status': 'success'}

In [3]:
import requests

url = "https://vuir.vu.edu.au/43558/1/Ep43558.pdf"

# Extract filename and append '.pdf'
filename = url.split("/")[-1]

# Full path where the file will be saved
path = os.path.join("articles", filename)

# Send a GET request to the URL with headers
response = requests.get(url, stream=True)

# Save the file if the request was successful
assert response.status_code == 200, response.status_code

with open(path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

# 1) Crop page


In [4]:
# Directory containing PDFs to process
pdfsDirectory = "articles"
pdfFiles = [
    f
    for f in os.listdir(pdfsDirectory)
    if os.path.isfile(os.path.join(pdfsDirectory, f))
]
totalFiles = len(pdfFiles)

# Output directory for cleaned text files
outputDirectory = os.path.join(pdfsDirectory, "")

# Crop all PDFs in the directory
cropAllPdfs(pdfFiles, outputDirectory, totalFiles)

# Test single PDF
# pdfPath= "review/1-s2.0-S0143416016300094-main.pdf"
# cropPDFMargins(pdfPath, outputDirectory)

Processing files:   0%|          | 0/1 [00:00<?, ?it/s]

document has 20 pages.


Processing files (Processed: 1/1):   0%|          | 0/1 [00:00<?, ?it/s]

Processing files (Processed: 1/1): 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]

Processing files (Processed: 1/1): 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]

In [5]:
os.makedirs("articles/cropped", exist_ok=True)
for filepath in glob.glob("articles/*_cropped.pdf"):
    filename = os.path.basename(filepath)
    shutil.copyfile(filepath, f"articles/cropped/{filename}")

In [6]:
os.listdir("articles/cropped")

['Ep43558_cropped.pdf']

# 2) Extract text

Table not working yet


In [7]:
# Directory containing your text files
text_files_directory = "articles/cropped/textOutput"

# Directory containing PDFs to process
pdfs_directory = "articles/cropped"

# Output directory for cleaned text files
output_directory = os.path.join(pdfs_directory, "textOutput")

# Process all PDFs in the directory
process_all_pdfs(pdfs_directory, output_directory)
process_text_files(text_files_directory)

Mining PDFs:   0%|          | 0/1 [00:00<?, ?it/s]

Ep43558_cropped.pdf


Mining PDFs: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]

Mining PDFs: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]

Cleaned text has been written to file: articles/cropped/textOutput/Ep43558_cropped_output_text.txt
Cleaning Ep43558_cropped_output_text.txt...
Ep43558_cropped_output_text.txt has been cleaned.


In [8]:
os.listdir("articles/cropped")

['Ep43558_cropped.pdf', 'textOutput']

# 3) Text segment

Get embeddings


In [9]:
# List of PDF file names
txt_directory = "articles/cropped/textOutput"
output_text_directory = "text_output"
# Configure OpenAI API key
key_file_path = "api_key"

if "OPENAI_API_KEY" in os.environ:
    with open(key_file_path, "w") as f:
        f.write(os.environ["OPENAI_API_KEY"])

process_files(
    txt_directory, output_text_directory, key_file_path, llm="Ollama"
)

Processing files:   0%|          | 0/1 [00:00<?, ?it/s]

Processing files: 100%|██████████| 1/1 [00:00<00:00,  1.23it/s]

Processing files: 100%|██████████| 1/1 [00:00<00:00,  1.23it/s]

Adding embeddings...


Saving data...
Data saved successfully.


# 4) Query text
